In [172]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics as mr

In [173]:
game=pd.read_csv('games.csv')

In [174]:
game.head()

,gameId,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,3326086514,1949,9,1,2,1,1,1,1,2,11,1,2,3,0,5,0,0,1,1
1,3229566029,1851,9,1,1,1,1,0,1,1,10,4,0,2,1,2,0,0,0,0
2,3327363504,1493,9,1,2,1,1,1,2,0,8,1,1,1,0,2,0,0,1,0
3,3326856598,1758,9,1,1,1,1,1,1,0,9,2,1,2,0,0,0,0,0,0
4,3330080762,2094,9,1,2,1,1,1,1,0,9,2,1,3,0,3,0,0,1,0


In [175]:
game.columns

Index(['gameId', 'gameDuration', 'seasonId', 'winner', 'firstBlood',
       'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon',
       'firstRiftHerald', 't1_towerKills', 't1_inhibitorKills',
       't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHeraldKills'],
      dtype='object')

# Model
Comparing Navie bayes / AODE /TAN tree

In [176]:
# Split train data and test data
train = game[0:int(0.9*len(game))]
test = game[int(0.9*len(game)):int(len(game))]

y_train = train['winner']
X_train = train.iloc[:,4:10]

y_test = test['winner']
X_test = test.iloc[:,4:10]

In [177]:
train.head()

,gameId,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,3326086514,1949,9,1,2,1,1,1,1,2,11,1,2,3,0,5,0,0,1,1
1,3229566029,1851,9,1,1,1,1,0,1,1,10,4,0,2,1,2,0,0,0,0
2,3327363504,1493,9,1,2,1,1,1,2,0,8,1,1,1,0,2,0,0,1,0
3,3326856598,1758,9,1,1,1,1,1,1,0,9,2,1,2,0,0,0,0,0,0
4,3330080762,2094,9,1,2,1,1,1,1,0,9,2,1,3,0,3,0,0,1,0


In [178]:
X_train

,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald
0,2,1,1,1,1,2
1,1,1,1,0,1,1
2,2,1,1,1,2,0
3,1,1,1,1,1,0
4,2,1,1,1,1,0
...,...,...,...,...,...,...
46336,2,2,2,0,1,1
46337,2,2,0,1,2,0
46338,1,1,1,1,2,0
46339,1,2,2,2,2,0


In [179]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=2.0)
clf.fit(X_train,y_train)

# test accurancy
clf.predict(X_test)
sum(clf.predict(X_test)==y_test)/len(y_test)

0.6766362400466109

In [180]:
# Average one dependent estimate
%run AODE.ipynb
Bs_aode = Aode()
Bs_aode.AodeTrain(X_train, y_train,[0,0,0,0,0,0])
Proba_aode = Bs_aode.aodepredict(X_test)
logProba_aode = Bs_aode.aodepredictLog(X_test)
yPredict_aode = np.argmax(logProba_aode, axis=1)+1 #because prediction is 0,1 and our label is 1,2
print(f"错误{sum(yPredict_aode!=y_test)}个，准确率为：{sum(yPredict_aode==y_test)/y_test.size}")

P(y,pa)训练完毕!
P(x|y,pa)训练完毕!
错误533个，准确率为：0.8964847543212274


In [181]:
# TAN tree
%run TAN.ipynb
Bs_Tan = Tan()
Bs_Tan.TanTrain(X_train, y_train,[0,0,0,0,0,0])
Proba_Tan = Bs_Tan.tanpredict(X_test)
logProba_Tan = Bs_Tan.tanpredictLog(X_test)
yPredict_Tan = np.argmax(logProba_Tan, axis=1)+1 #because prediction is 0,1 and our label is 1,2
print(f"错误{sum(yPredict_Tan!=y_test)}个，准确率为：{sum(yPredict_Tan==y_test)/y_test.size}")

P(y)训练完毕!
P(x|y,pa)训练完毕!
错误541个，准确率为：0.8949310545737036


In [182]:
Bs_Tan.CMI_dict

{(0, 1): 0.0555752193081815,
 (0, 2): 0.027394351962594647,
 (0, 3): 0.014731630725936267,
 (0, 4): 0.04127659594207565,
 (0, 5): 0.011257311955587228,
 (1, 2): 0.07295418997732407,
 (1, 3): 0.047133368305087894,
 (1, 4): 0.10342289048297496,
 (1, 5): 0.04228430420303731,
 (2, 3): 0.11201582813621128,
 (2, 4): 0.05729804290420339,
 (2, 5): 0.017505084449420925,
 (3, 4): 0.04286557518830103,
 (3, 5): 0.015291053794174836,
 (4, 5): 0.023384127008345017}

In [183]:
Bs_Tan.f_relationship

{3: 2, 4: 1, 0: 1, 5: 1}

In [184]:
#Logistics regression
from sklearn.linear_model import LogisticRegression

lrclf = LogisticRegression()
lrclf.fit(X_train, y_train)

# test accurancy
lrclf.predict(X_test)
sum(lrclf.predict(X_test)==y_test)/len(y_test)

0.8314235773936687

In [185]:
# Logistics regression with Interactions
X_train.loc[:,'BloodTower'] = X_train.loc[:,'firstBlood'] * X_train.loc[:,'firstTower']
X_train.loc[:,'DragonTower'] = X_train.loc[:,'firstDragon'] * X_train.loc[:,'firstTower']

C:\Users\zhziy\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [186]:
X_test.loc[:,'bloodtower'] = X_test.loc[:,'firstBlood']*X_test.loc[:,'firstTower']
X_test.loc[:,'DragonTower'] = X_test.loc[:,'firstDragon']*X_test.loc[:,'firstTower']

In [187]:
lrclf = LogisticRegression()
lrclf.fit(X_train, y_train)

# test accurancy
lrclf.predict(X_test)
sum(lrclf.predict(X_test)==y_test)/len(y_test)

0.8222955913769664